# Examen – Modelos de Ensamble: Árboles y Boosting

**Instrucciones**: Justifica cada respuesta de manera clara. Usa fórmulas donde aplique y sé específico en tus argumentos.

---

## Sección I: Árboles de decisión (15 puntos)

### 1. (5 pts)  
**Explica cómo se construye un árbol y qué criterio usa para decidir los splits. Explica tanto para el caso de clasificación como de regresión.**

Un modelo el cual utiliza varios niveles, donde en cada nivel existe una caractrística que se evalua a través de un umbral, va dividiendo los datos en los que son mayor o menor al umbral definido en la característica.


### Regresión

Este algoritmo usa todas las variables y los splits posibles (umbrales) y elige el umbral en cada variable que más reduzca la varianza.


$$
\text{Reducción de varianza} = \text{Varianza total} - \left( \frac{N_1}{N} \times \text{Var}(G1) + \frac{N_2}{N} \times \text{Var}(G2) \right)
$$

Varianza de cada rama ponderada por el número de gente de cada una de las ramas.

### Clasificación

De igual forma que en la regresión, el modelo usa todas las variables y prueba todos los umbrales posibles, eligiendo el que cause una mayor reducción de impureza, la cual se calcula a través del Gini o de la Entriopía.

$$
Gini = 1 - \sum p_i^2
$$

$$
Entropía = -\sum p_i \log_2 p_i
$$


$$
\text{Ganancia de Impureza} = \text{Impureza Nodo Padre} - \left( \frac{N_1}{N} \times \text{Impureza}(G1) + \frac{N_2}{N} \times \text{Impureza}(G2) \right)
$$

Impureza de cada rama ponderada por el número de personas en cada una de ellas.

### 2. (5 pts)  
**Da un ejemplo de sobreajuste en un árbol de decisión. Explica cómo se podría evitar sin necesidad de usar ensambles.**

Un caso común es cuando el max_depth es muy grande, ya que el árbol se va expandiendo cada vez más haciendo muchos niveles y muchos splits lo cual provoca que termine memorizando los datos. La forma más sencilla de evitar esto es con un cross validation que pruebe distintos depths de árbol para encontrar el mejor, además de mover otros hiperparámetros.

### 3. (5 pts)  
**Si te fijas, en clase nunca hicimos escalamiento (`StandardScaler`). ¿Por qué los árboles no lo necesitan?**

El `StandardScaler` sirve para poner en una misma escala tus variables, esto es útil para modelos de KNN que usan distancias, y puede ser útil en regresiones para dar como la misma importancia a las variables, sin embargo, por la construcción de los árboles esto no es necesario ya que comparas sobre el valor real de tus datos para definir tus reglas de decisión (umbrales) que son los que más reducen la varianza. Al hacer splits sobre tus valores no es de ayuda escalarlas ya que con los valores originales se pueden establecer los splits adecuados.

---

## Sección II: Random Forest (20 puntos)

### 4. (10 pts)  
**Explica cómo funciona un Random Forest. ¿En qué se basa? ¿Por qué es una buena idea?**

Utilizas la técnica de bootsrap para hacer muestreos aleatorios de tus datos, con cada uno de los muestreos entrenas un árbol de decisión distinto logrando así un buen entrenamiento con la construcción de múltiples árboles de decisión. Cada uno de estos modelos elige distintas variables de forma aleatoria, realiza sus predicciones, y el promedio de estas sería la predicción utilizada por el algoritmo de Random Forest o la más usada en caso de clasificación. Esto es útil ya que de esta forma puedes probar muchos datos distintos por el muestreo aleatorio para entrenar de mejor manera, además de utilizar distintas variables y splits, esto ayuda a dar mejores resultados que un solo árbol pues hace un entrenamiento más robusto.

### 5. (10 pts)  
**Menciona dos ventajas y dos desventajas del Random Forest comparado con un solo árbol. Sé específico, no generalices.**

`Ventajas`:
- Menos probabilidad de hacer overfitting por el uso de bootsrap ya que añada aleatoriedad y usa todos los resultados.
- Mejores predicciones ya que combina los resultados me múltiples modelos entrenados con una gran variedad de datos.

`Desventajas`:
- Computacionalmente costoso, es decir, tarda en correr por entrenar muchos árboles.
- Su interpretación no es ta rápida y directa como la de un árbol de decisión.

---

## Sección III: Gradient Boosting (25 puntos)

### 6. (10 pts)  
**Explica, paso a paso, cómo funciona el algoritmo de **`Gradient Boosting`**. Incluye el concepto de residuales y cómo se minimiza la pérdida en cada iteración.**

Inicias tu modelo con un valor constante (puede ser el promedio de tus datos por ejemplo).

Después calculas los residuales de este modelo con un loop a través de los árboles (valor real - valor predicho), de manera formal:

$$
r_{im} = - \left[ \frac{\partial L(y_i, F(x_i))}{\partial F(x_i)} \right]_{F(x)=F_{m-1}(x)} \quad \text{para } i = 1 \dots n
$$

Una vez hecho esto se ajusta un árbol de regresión para predecir los residuales y para cada una de las regiones obtienes:

$$
\gamma_{jm} = \arg\min_\gamma \sum_{x_i \in R_{jm}} L(y_i, F_{m-1}(x_i) + \gamma)
$$

Por último se actualiza el modelo que es el modelo anterior más el nuevo árbol ajustado por el learning rate:

$$F_m(x) = F_{m - 1}(x) + \nu (\text{nuevo árbol})$$

El proceso se repite hasta converger, haciendo el error cada vez más chico. Similar al descenso en gradiente.



### 7. (15 pts)  
**¿Cuál es la diferencia entre Gradient Boosting y Random Forest en términos de cómo combinan los árboles?** 

Random Forest usa muetreos aleatorios para entrenar muchos árboles distintos y usa las predicciones de cada uno para la predicción final, ya sea el promedio y los más usada. Gradient Boosting obtiene los residuales (real - predicho) y ajusta un árbol de regresión para poder predecir los residuales, el promedio de estos residuales se usan como predicción para el siguiente árbol y actualiza el modelo usando el anteriror más el nuevo árbol multiplicado por un ajuste.

Por lo tanto funcionan muy distinto ya que uno hace muchos árboles y promedia las predicciones (Random Forest) y otro es más como descenso en gradiente donde usa los residuales y su predición de estos para actualizar el modelo.

---


## Sección IV: XGBoost (20 puntos)

### 8. (10 pts)  
**Explica cómo XGBoost optimiza el proceso de boosting usando una expansión de Taylor de segundo orden.**

La función de pérdida se compone de dos partes, una que son los árboles anterirores y otra parte que representa el árbol actual. La parte de los árboles anteriores se usa cmo constante (ya esta dado) y se optimiza lo que aporta el nuevo árbol, llegando a:

$$
\mathcal{L}^{(t)} =
\sum_{i=1}^n l(y_i, \hat{y}_i^{(t-1)} + f_t(x_i)) + \sum_{k=1}^{t} \Omega(f_k)
$$

$\Omega(f_k)$ es un factor de regularización el cual castiga árboles más complejos.

Como la función de pérdida es compleja, se utiliza las series de Taylor para poder aproximar la función de pérdida cerca de las predicciones. La serie de Taylor de segundo orden permite aproximar la función de pérdida con una forma cuadrática, la cual es más sencilla de optmizar y vuelve más eficiente el proceso. Utilizando esta expansión, el gradiente y el hessiano la función de pérdida queda de la siguiente forma:

$$
\tilde{\mathcal{L}}^{(t)} = \sum_{i=1}^n \left[ g_i f_t(x_i) + \frac{1}{2} h_i f_t(x_i)^2 \right] + \Omega(f_t)
$$

Con ello la optimización es un problema cuadrático más sencillo de optimizar. También lo vuelve eficiente y útil la aparición de $f_t(x_i)$, ya que es la predicción del nuevo árbol, esta sale del factor $(x-a)$ de la expansión de Taylor.

### 9. (5 pts)  
**¿Qué es el *similarity score*? ¿Qué es el *output value*? ¿De dónde salen estas fórmulas y cuál es su interpretación?**

### Output Value

Es el valor que predice una hoja del árbol, cuando lleguen los datos a esa hoja el output value es el valor arrojado.

Una vez hecha la expansión de Taylor y que se agrupando los árboles como funciónes pieza por pieza, se tiene la función de pérdida:

$$
\mathcal{L}^{(t)} \approx \sum_{j=1}^{T} \left[
G_j w_j + \frac{1}{2} (H_j + \lambda) w_j^2
\right] + \gamma T
$$

Minimizando esto por hoja $w_j$, es decir, derivando e igualando a cero se llega a:

$$
\text{Output value} = w_j^* = -\frac{G_j}{H_j + \lambda}
$$

Por lo que este output value es el valor óptimo arrojado que minimiza la función de pérdida.

### Similarity Score

El similarity score nos dice cuanto mejora la pérdida en cada una de loas hojas (nos dice la calidad de la hoja):

La función de pérdida por hoja es:

$$
\mathcal{L}_j(w_j) = G_j w_j + \frac{1}{2} (H_j + \lambda) w_j^2
$$

Sustituyes el valor óptimo de $w_j$ el cual minimiza la funciónde pérdida calculada por la función de Taylor y llegas a:

$$
\text{Similarity Score} = \frac{G_j^2}{H_j + \lambda}
$$

Con esta fórmula calculas la mejora de l a pérdida y te ayuda a encontrar splits.

### 10. (5 pts)  
**XGBoost y otros modelos de gradient boosting permiten evaluar la importancia de las variables con diferentes métricas: `weight` y `gain`. Explica qué representa cada una. ¿Cuál crees que es más útil para interpretar un modelo y por qué?**

El `weight`nos dice cuantas veces fue utilizada una variable en todos los árboles del modelo, es simplemente su frecuencia. Por otro lado el `gain` nos dice que tanto aporta esa variable en la reducción de varianza del modelo (cuanto minimiza la pérdida).

Da más información el `gain`de una variable, ya que el `weight`solo te dice cuanto se uso, más no si ayudo usar o no la variable, mientras que el gain es un indicador directo de que tanto influye una variable en las predicciones del modelo, lo cual es valioso para analizar tus datos y saber cuales sirven y cuales no.

---

## Sección V: LightGBM y CatBoost (15 puntos)

### 10. (5 pts)  
**Explica qué es **histogram-based splitting** y cómo lo implementa LightGBM para ganar velocidad.**

Habitualemnte los árboles buscan hacer el split óptimo usando todos los valores para una variable lo cual usa muchos datos y hace más lento el entrenamiento de los modelos (tarda más en correr). Lo que hace a lightgbm más eficient es dividr en bins los datos (250 normalmente), esto genera histogramas de los datos que se tienen y al hacer el entrenamiento se busca el mejor split entre bins, usando así menos datos para encontrar los splits lo cual agiliza el proceso de entrenamiento miemtras mantiene muy buenos resultados.

### 11. (5 pts)  
**¿Qué problema específico resuelve CatBoost respecto al manejo de variables categóricas? ¿Cómo lo hace?**

Existe el label encoding donde se le asigna un número a cada variable, esto causa problemas ya que el modelo piensa que una categoría vale más que otra (gato>perro). Tambien está el target encoding que usa el valor promedio de cada categoría pero esto puede provocar overfitting si se tiene categorías pequeñas (3 cosas diferentes).

Estos problemas los resuelve catboost usando su propio encoding de variables categóricas.

$$ encoding = \frac{\sum y  + a * \mu}{n + a} $$

Esto lo que hace es acomodar de forma aleatoria las variables y calcula su promedio de forma secuencial. Usa el promedio conocido hasta ese momento. Esto lo hace en múltiples permutaciones del dataset calculando el valor del encoding en cada iteración y finalmente promedia los resultados, logrando así un encoding estable.

### 12. (5 pts)  
**Compara LightGBM y CatBoost: ¿cuándo usarías uno sobre el otro? Sé claro y justifica en base a tipo de datos, velocidad o precisión.**

El uso de uno u otro modelo depende de tus objetivos y de las carcaterísticas de tus datos.

Si se tiene un dataset mediano o chico catboost es buena opción, mientras que en dataset muy grandes con muchas variables numéricas es mejor lightgbm por precisión y velocidad. Si en el dataset se tiene muchas variables categóricas la mejor opción es catboost pues da los mejores resultados. Si se busca un entrenamiento y rápido y con alta precisión lightgbm sería más adecuado. Por último si necesitas buen rendimiento sin mucho tuning del modelo (hiperparámetros) catboost suele dar buenos resultados.

## Sección VI: Power Analysis (5 puntos)

### 13. (5 pts)  
**¿Qué es un *power analysis* y para qué sirve? ¿En qué contexto lo hemos usado en clase y por qué es importante antes de correr un experimento?**

Es una técnica estadística que ayuda a encontrar los tamaños de muestra necesarios para que los resultados obtenidos tengan significancia estadísticas, además de darnos la probabilidad de que el experimento realizado encuentre un efecto real (estadísticamente significativo). En la clase se uso el ejemplo de una página web y el color de sus botines para ver si tenía un impacto cambiar el color, se obtuvo tamaño de muestra y probabilidad de significancia estadística. Además se comento como puede ser usado por páginas como Amazon y Kueski y como sería útil en contexto de otorgar créditos.